# Interdisciplinary Project
by Marina Sommer | 11778902

In [17]:
from itertools import product
crawl_names=['CC-MAIN-2013-20']
top_lvl_domains=['de', 'at']
target_words=['angreifen', 'anfassen', 'anlangen']
year_tld = list(product(crawl_names, top_lvl_domains))
for year, tld in year_tld:
    print(year, tld)

CC-MAIN-2013-20 de
CC-MAIN-2013-20 at


In [15]:
def jaccard_similarity(list1, list2):
    set1, set2 = set(list1), set(list2)  # Convert to sets if they are lists
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

In [13]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union if union != 0 else 0

In [16]:
list1 = ['IOM', 'Angriff', 'tätlich', 'niederschießen', 'beziehen', 'angriffen']
list2 = ['IOM', 'Angriff', 'fliehen', 'abschießen', 'beziehen', 'beschießen',]
# JI = intersection over union (IoU)
# intersection = 3
# union = 6 + 6 - 3 = 9
# JI = 3/9 = 1/3 = 0.33333
jaccard_similarity(list1, list2)

0.3333333333333333

Workflow is as follows:

In [1]:
# import packages
from urllib.request import urlretrieve
import os
import re 
import random
import gzip
import json
from collections import Counter
import time
from warcio import ArchiveIterator
import requests
import pandas as pd
import numpy as np
import gensim
import spacy

In [9]:
# test path
import os
import glob
crawl_name = 'CC-MAIN-2013-20'  # take a small crawl for testing
top_lvl_domain = 'at'
crawl_dir = os.path.join("S:", "msommer", crawl_name, top_lvl_domain)
#os.chdir(crawl_dir)
key = "crawl-data/CC-MAIN-2013-20/segments/1368708808767/wet/CC-MAIN-20130516125328-00057-ip-10-60-113-184.ec2.internal.warc.wet.gz"
key_path = "CC-MAIN-20130516121618-00013-ip-10-60-113-184.ec2.internal.warc.wet"
files = glob.glob("*.warc.wet.gz")
files

['CC-MAIN-20130516121618-00013-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516101433-00040-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516103603-00068-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516105732-00041-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516110654-00064-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516112133-00074-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516130024-00038-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516123613-00087-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516092622-00047-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516095809-00080-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516131926-00033-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516095504-00038-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516101218-00034-ip-10-60-113-184.ec2.internal.warc.wet.gz',
 'CC-MAIN-20130516112715-

In [11]:
from gensim.models import Word2Vec
crawl_name = 'CC-MAIN-2013-20'  # take a small crawl for testing
top_lvl_domain = 'at'
crawl_dir = os.path.join("S:", "msommer", crawl_name, top_lvl_domain)
# load model
model_fname = os.path.join(crawl_dir, "word2vec.model")
model = Word2Vec.load(model_fname)

# save word sets of target words
target = 'angreifen'
w1 = model.wv.most_similar(target, 10)
w2 = model.wv.similar_by_word(target, 10)
print(w1)
print(w2)

[('schimpfen', 0.4794471263885498), ('Papa', 0.46053141355514526), ('verzeihen', 0.45761623978614807), ('Sagen', 0.44254419207572937), ('abstempeln', 0.4383883476257324), ('entgegnen', 0.43529683351516724), ('Putin,', 0.4345760643482208), ('aufhören', 0.426441490650177), ('Dummheit', 0.419924259185791), ('nachdenken', 0.41940540075302124)]
[('ausdrücken', 0.5550252795219421), ('umbringen', 0.5167557001113892), ('rassistisch', 0.5018532872200012), ('abstempeln', 0.5010532140731812), ('verzeihen', 0.49874329566955566), ('irren', 0.4904670715332031), ('Putin,', 0.4881359338760376), ('mitmachen', 0.48637640476226807), ('Marionette', 0.48224660754203796), ('beleidigt', 0.48221421241760254)]


In [1]:
from gensim.test.utils import common_texts
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [7]:
from HanTa import HanoverTagger as ht
tagger = ht.HanoverTagger('morphmodel_ger.pgz')
mails=['Hallo. Ich spielte am frühen Morgen und ging dann zu einem Freund. Auf Wiedersehen',
       'Guten Tag Ich mochte Bälle und will etwas kaufen. Tschüss']
mails_lemma = []
for mail in mails:
    print(mail.split())
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    print(lemma)
    mails_lemma.append(' '.join(lemma))
    print(mails_lemma)
    break

['Hallo.', 'Ich', 'spielte', 'am', 'frühen', 'Morgen', 'und', 'ging', 'dann', 'zu', 'einem', 'Freund.', 'Auf', 'Wiedersehen']
['hallo.', 'ich', 'spielen', 'an', 'früh', 'Morgen', 'und', 'gehen', 'dann', 'zu', 'ein', 'Freund.', 'auf', 'Wiedersehen']
['hallo. ich spielen an früh Morgen und gehen dann zu ein Freund. auf Wiedersehen']


In [5]:
import time
t = time.time()
time.sleep(1)
print("Execution ran for", round((time.time() - t) / 60, 2), "minutes.")

Execution ran for 0.02 minutes.


In [2]:
# 1) select crawl: CC-MAIN-2023-40 and download cluster.idx
crawl_name = "CC-MAIN-2023-40"
top_lvl_domain = "at"   # oder "de"
path1 = 'https://data.commoncrawl.org/cc-index/collections/'
path2 = '/indexes/'
path_ccrawl = path1 + crawl_name + path2
url = path_ccrawl + 'cluster.idx'
print(url)

https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2023-40/indexes/cluster.idx


In [3]:
crawl_dir = os.path.join("S:", "msommer", crawl_name, top_lvl_domain)
if not os.path.exists(crawl_dir):
   os.makedirs(crawl_dir)
cluster_file = os.path.join(crawl_dir, "cluster.txt")
#urlretrieve(url, cluster_file)

In [4]:
# 2) filter cdx files with tld '^at,' or '^de,'
regex = '^' + top_lvl_domain + ','
with open(cluster_file, "rt") as file:     
    cdx_files = []
    for line in file:
        tld = line.split("\t")
        match = re.search(regex, tld[0])  # ^at,
        if match: 
            if not tld[1] in cdx_files:
                cdx_files.append(tld[1])
print(cdx_files)      

['cdx-00002.gz', 'cdx-00003.gz', 'cdx-00004.gz']


In [5]:
# 3) randomly choose cdx files, e.g. cdx-00003.gz -> way too much for de
if len(cdx_files) > 1:
    idx = random.sample(range(0, len(cdx_files)), 1)
    print(idx)
    cdx_files = [cdx_files[i] for i in idx]
    print(cdx_files)

[1]
['cdx-00003.gz']


In [6]:
# 4) download cdx files (one takes ~ 2 min)
# wget “https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2022-05/indexes/$i.gz"
for file in cdx_files:
    print(file)
    url = path_ccrawl + file
    filename = os.path.join(crawl_dir, file)
    #urlretrieve(url, filename)

cdx-00002.gz
cdx-00003.gz
cdx-00004.gz


In [5]:
from urllib.request import urlretrieve
def tryDownload(url, filename, retries=0):
    if retries > 10: 
        print("Download failed.")
        return
    try:
        urlretrieve(url, filename)  
    except:
        time.sleep(1)
        tryDownload(url, filename, retries+1)

In [6]:
# 5) gunzip and open it - one line looks like the following:
warc_files = []
for file in cdx_files:
    filename = os.path.join(crawl_dir, file)
    print(filename)
    with gzip.open(filename, 'rt') as f:
        for line in f:          
            match = re.search(regex, line)  # ^at,
            if match: 
                l = line.split("{\"url\":")
                string = "{\"url\":" + l[1]
                d = json.loads(string)
                warc = d["filename"]
                if int(d["length"]) > 10000: 
                    warc_files.append(warc)
        count_dict = Counter(warc_files).most_common()    # 1 - 73 occurrences per warc file
        
        # only download warc files with a lot of occurrences --> maybe download not necessary? --> see below
        wet_files = []
        for key, value in count_dict.items():
            if value >= 50:
                # 6) get filename, convert to wet file and download (WARC: ~ 2 min, WET: 15 sec )
                key = key.replace("/warc/", "/wet/").replace("warc.gz", "warc.wet.gz")
                key_path = key.split("/")[-1]
                filename = os.path.join(crawl_dir, key_path)
                wet_files.append(filename)
                print(filename)
                url = "https://data.commoncrawl.org/" + key
                print(url)
                #tryDownload(url, filename)  
                break
                
    
            
    break

S:msommer\CC-MAIN-2023-40\at\cdx-00003.gz


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



AttributeError: 'list' object has no attribute 'items'

warcinfo
request
response
metadata

WARC/1.0
WARC-Type: conversion
WARC-Target-URI: http://ah.zzjiahe.net/news949196.html
WARC-Date: 2023-10-01T04:54:41Z
WARC-Record-ID: <urn:uuid:02e7a2be-bcf6-47d6-ae05-30660a907a1f>
WARC-Refers-To: <urn:uuid:de36273e-b04b-4de1-b759-0d74586b1ec9>
WARC-Block-Digest: sha1:JBRJCLDEN7QUUHDWBNQ746KQ4VJGVAJ6
WARC-Identified-Content-Language: zho
Content-Type: text/plain
Content-Length: 3173

In [35]:
import string
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('german')
def count_pct_and_stopwords(text, stopwords):
    pct_count = 0
    sw_count = 0
    line_tok = nltk.word_tokenize(text)

    for tok in line_tok:
        if tok in string.punctuation or tok == '\n':
            pct_count += 1
        if tok in stopwords:
            sw_count += 1
            
    lb_count = text.count('\n')

    return pct_count, sw_count, lb_count

In [46]:
wet_file = "S:msommer\CC-MAIN-2013-20\\at\CC-MAIN-20130516112802-00088-ip-10-60-113-184.ec2.internal.warc.wet.gz"
fname = wet_file.replace(".warc.wet.gz", "-text.txt")
with open(wet_file, 'rb') as stream, open(fname, 'wt', encoding='utf-8') as f:
    x = 0
    for record in ArchiveIterator(stream):
        if record.rec_type == 'conversion':
            regex = '\.' + top_lvl_domain + '/'
            match = re.search(regex, record.rec_headers.get_header('WARC-Target-URI'))
            length = int(record.rec_headers.get_header('Content-Length'))
            rec_type = record.rec_headers.get_header('Content-Type')
            #lang = record.rec_headers.get_header('WARC-Identified-Content-Language') # not available

            if match and length > 10000 and rec_type == "text/plain":
                print(record.rec_headers.get_header('WARC-Target-URI'))
                content = record.content_stream().read().decode('utf-8', errors='replace') # errors='surrogateescape' 
                pct_cnt, sw_cnt, lb_cnt = count_pct_and_stopwords(content, stop_words)
                print(pct_cnt, sw_cnt, lb_cnt)
                if sw_cnt == 0:
                    continue
                elif pct_cnt / sw_cnt > 1 or lb_cnt / sw_cnt > 0.5:
                    continue
                f.write(content)
                x += 1
        if x > 10:
            break


http://derstandard.at/1216917895853/Obamania--Alles-nur-ein-Hype
426 830 6
http://derstandard.at/1334796665270/Mageres-Paszek-Comeback
190 439 15
http://derstandard.at/1350259722629/Pflaster-Abloesen-ohne-Schmerz
268 536 4
http://derstandard.at/plink/1358304904279?_pid=29953867
448 1068 10
http://www.bmeia.gv.at/aussenministerium/aussenpolitik/auslandskultur/veranstaltungsliste-weltweit.html?so=1&sb=2&pgpt=13&dd=18&dm=11&dy=2012&sp=1&act=1&no_cache=1
1172 93 466
http://www.groupon.at/deals/online-deal/isimax/4985812?jGroupSelector=true&STUVWX=true
210 348 213
http://www.kleinezeitung.at/steiermark/graz/graz/1569542/index.do
149 292 418
http://www.ornauer.at/lebensfreude-blog/category/metaphern/
963 1022 360
http://www.thalia.at/shop/kundenempfehlungen-musik/rezartikel/der_alte_koenig_in_seinem_exil/arno_geiger/ISBN3-446-23690-2/ID25998269.html
476 1275 609
http://www.wienmuseum.at/de/besucherinformationen/kataloge.html?L=1&fsize=0&tx_wxcatalogue_pi1[catalogue_id]=149
2231 1225 1582
htt

In [ ]:
### QUESTION: Wie löse ich das Problem, dass oft nur Überschriften im Text vorkommen?! 
# -> vlt beim einlesen von text file line per line die mit wenigen zeichen vernachlässigen
### QUESTION: Wie stelle ich sicher, dass nur deutsche Webinhalte verwendet werden??
# -> line per line language detection ?? -> sehr zeitaufwendig

Ausgezeichnet mit dem H�lderlin-Preis 2011�Arno Geiger hat ein tief ber�hrendes Buch �ber seinen Vater geschrieben, der trotz seiner Alzheimerkrankheit mit Vitalit�t, Witz und Klugheit beeindruckt. Die Krankheit l�st langsam seine Erinnerung und seine Orientierung in der Gegenwart auf, l�sst sein Leben abhandenkommen. Arno Geiger erz�hlt, wie er nochmals Freundschaft mit seinem Vater schlie�t und ihn viele Jahre begleitet. In nur scheinbar sinnlosen und oft so wunderbar poetischen S�tzen entdeckt er, dass es auch im Alter in der Person des Vaters noch alles gibt: Charme, Witz, Selbstbewusstsein und W�rde. Arno Geigers Buch ist lebendig, oft komisch. In seiner tief ber�hrenden Geschichte erz�hlt er von einem Leben, das es immer noch zutiefst wert ist, gelebt zu werden.

-> Umlautfehler leider schon in der Quellendatei vorhanden (siehe oben)!

In [ ]:
from langdetect import detect
            if line.count('.') < 3 or len(line) < 100:
                continue
            try:
                lang = detect(line)
            except:
                lang = "none"
            if lang != "de":
                continue

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e11778902\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
fname = "S:msommer\CC-MAIN-2013-20\\at\CC-MAIN-20130516112802-00088-ip-10-60-113-184.ec2.internal-text.txt"
final_fname = fname.replace("text.txt", "FINAL.txt")
min_stopwords = 10
with open(final_fname, 'wt', encoding="utf-8") as f:
    #for fname in glob.glob("*-text.txt"):
    iter = 0
    with open(fname, "rt", encoding="utf-8", errors='replace') as file:
        for line in file:
            # if a line consists of several stopwords, it is considered as real text
            # this is also kind of language detection
            #check_sw = not any(word for word in line.split() if word in stop_words)
            sw_cnt = sum(word in stop_words for word in line.split())
            comma_cnt = line.count(',')
            if sw_cnt < min_stopwords or line.count('.') < 3 or len(line) < 100 or comma_cnt/sw_cnt > 1:
                continue
            f.write(line)
            print(line)
            iter += 1
            
            #print(len(line))
            #if iter > 35:
             #   break
            

Obamania - Alles nur ein Hype? - FS-Misik - derStandard.at › MeinungderStandard.at›Meinung›Blogs›Politik›FS-MisikSeite1InternationalInlandWirtschaftWebSportPanoramaEtatKulturWissenschaftGesundheitBildungReisenLifestyleFamilieMeinungKommentareKolumnenKommentare der anderenGastkommentareLeserkommentareBlogsPolitikWirtschaftGesellschaftdieStandard.atdaStandard.atZuZweitPartnersucheGebrauchtwagenAutoMobilAutoMotorradRadGebrauchtwagenAutopreiseMeine InserateSuchagentenImmobilienImmobilien suchenGewerbeimmobilienMeine InserateSuchagentenMerklisteEinstellungenHilfeKarriereJob & KarriereManagementJobsuche & StellenmarktKarrierepoolSuchagentKundencenterFirmen A - ZsuchenWetter Wienanmeldenabmelden PostingsMitposterInnen, die ich lese:Alle meine PostingsaktualisierenExpressNachrichten in aller KürzeCommunityAlles zur CommunityNewsletterNachrichten, die zu Ihnen kommen: Newsletter, Feeds und SMSMobilAlles zu unseren mobilen Angeboten: Apps, Mobilversion und SMSTV/RadioUnsere Radio- und TV-Angebot

https://stackoverflow.com/questions/34612691/how-can-one-extract-every-payload-from-warc-wet-gz

The code extracts the first 100 text "payloads" from the .warc.wet.gz file. The 100 created text files have shortened names which correspond to their associated url in the warc.wet.gz file. Many of the urls are too long to be used as as convenient names. The file wet_list_out.txt is generated and provides the correspondence between the shortened .txt file names and the associated urls.
The code works. Of course you need to replace warc_wet_gz with the full path name of your own .warc.wet.gz file. If you want all the payloads, not just the first 100, set num to a very large number. Variable num was tacilty assumed to be 100 in the initial post. Why was it rated as not useful? The problem is solved. If you find issues with it please make specific suggestions. 

In [ ]:
import requests
from warcio.archiveiterator import ArchiveIterator
from requests_file import FileAdapter
def print_f_records(fpath,f_out,num):
    record_count = 0
    conversion_count = 0
    header_count = 0
    splitter = "://"
    mx = num
    s = requests.session()
    s.mount('file://',FileAdapter())
    resp = s.get(fpath, stream=True)
    for record in ArchiveIterator(resp.raw, arc2warc=True):
        record_count +=1
        if record.rec_type == 'warcinfo':
            print(record.raw_stream.read())
        elif record.rec_type == 'conversion':
            conversion_count +=1
            if record.raw_stream.read is not None:
                if record.rec_headers.get_header('Content-Type') == 'text/plain':
                    header_count +=1
                    if header_count > mx:
                        continue
                    print("\n text/plain header no: ", header_count)
                    prefix = f"a{header_count}_"        # filenames will begin with a<header_count>
                    fname1 = (record.rec_headers.get_header('WARC-Target-URI'))
                    fname = fname1.split(splitter)[1]   # get the name part which follows ://
                    fname = prefix + fname[:22] + '.txt'    # keep entire file name to 32 characters or less
                    fname = fname.replace("/", "_")
                    content = record.content_stream().read()
                    f = open(fname, 'wb')               # note the wb; write in binay mode because the content is characters
                    f.write(content)
                    f.close()
                    line = f'{fname1},{fname}' + "\n" # keep a record of the url translations
                    f_out.write(line)
                    print("created text file: ", fname)
                    if header_count == mx:
                        print(f"\n... completed {mx} txt files ... \n")
    print("Number of records: ", record_count)
    print("Number of conversion types: ", conversion_count)
    print("Number of text/plain http_headers: ", header_count)
    print("\nFINISHED")
warc_wet_gz ='file:////home/psl/CCrawl/text_extraction/text_from_wet/CC-MAIN-20230928063033-20230928093033-00593.warc.wet.gz'
print(f'\n warc.wet.gz file is {warc_wet_gz}')
num=100
f_out=open("wet_list_out.txt", 'w')
print_f_records(warc_wet_gz, f_out, num)
f_out.close()

In [19]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e11778902\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [28]:
line = "Das ist ein Beispiel ? Satz für ?? viele Anwendungen, wie der Zählung von Funktionswörtern!"
pct_cnt, sw_cnt = count_pct_and_stopwords(line, stop_words)
print(pct_cnt, sw_cnt)
if pct_cnt / sw_cnt > 1:
    print("rm!")

5 6


In [ ]:
    nlp = spacy.load("de_core_news_sm")
    words = set(nlp.vocab.strings)
    word = 'anlangen'
    print(f"Is '{word}' a German word: {word in words}")